In [10]:
# Import pandas
import pandas as pd
import numpy as np
from gensim.corpora import Dictionary
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import chi2  as chi2
from scipy.stats import chi2 as chi2stat

from sklearn.feature_selection import mutual_info_classif

In [11]:
# Assign spreadsheet filename to `file`
file = '../../data/Mozilla_4source.xlsx'
# Load spreadsheet
xl = pd.ExcelFile(file)
# Load a sheet into a DataFrame by name: df1
df = xl.parse('total')

In [12]:
print('Total: ', len(df))

Total:  2635


In [43]:
class NaiveBayesClassifer:
    def __init__(self):
        self.name = 'Naive Bayes Classifier'
    
    def train (self, X_train, y_train, number_of_unique_words = 100000):
        self.prior = np.zeros(5)
        labels = list(y_train)
        
        for i in range(5):
            self.prior[i] = labels.count(i)/len(labels)
        
        self.dictionary = Dictionary()

        summaries = list(X_train.loc[:,'description'])
        for summary in summaries:
            self.dictionary.add_documents([summary.split()])
        
        self.number_of_unique_words = len(self.dictionary)
        print('Number of unique words:', self.number_of_unique_words)
        self.summary_feature = np.ones((len(X_train), self.number_of_unique_words))
        self.summary_feature_selection = np.zeros((len(X_train), self.number_of_unique_words))
        self.summary_word_count = np.ones((5, self.number_of_unique_words))
        
        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        for i in range(len(summaries)):
            for j in range(len(summaries[i])):
                self.summary_feature[i][summaries[i][j]] += 1
                self.summary_feature_selection[i][summaries[i][j]] += 1
                self.summary_word_count[labels[i]][summaries[i][j]] += 1
        
        for i in range(5):
            self.summary_word_count[i] /= np.sum(self.summary_word_count[i]) 
        
        self.important_features = []
        
        for i in range(self.summary_feature.shape[1]):
            number_of_unique_values = len(np.unique(self.summary_feature_selection[:, i]))
            mutual_information = mutual_info_classif(self.summary_feature_selection[:, i].reshape(-1, 1), np.array(labels))
            chicritval = chi2stat.ppf(.99, number_of_unique_values - 1) / (2 * self.summary_feature.shape[0] * np.log(2))
            if (mutual_information > chicritval):
                self.important_features.append(i)
            
        del self.summary_feature_selection
        
        print('Total number of features:', self.number_of_unique_words)
        print('Number of selected features:', len(self.important_features))
        
    def predict (self, X_test):
        y_test = np.ones((X_test.shape[0], 5))
        
        '''
        for i in range(X_test.shape[0]):
            for j in range(5):
                y_test[i][j] = (self.prior[j])
        '''
        
        summaries = list(X_test.loc[:,'description'])
        
        for i in range(len(summaries)):
            summaries[i] = self.dictionary.doc2idx(summaries[i].split())
        
        for i in range(X_test.shape[0]):
            for lab in range(5):
                log_sum_summary = 0
                for j in range(len(summaries[i])):
                    if (summaries[i][j] != -1 and summaries[i][j] in self.important_features and self.summary_word_count[lab][summaries[i][j]]):
                        log_sum_summary += np.log(self.summary_word_count[lab][summaries[i][j]])
        
                #y_test[i][lab] = np.log(y_test[i][lab])
                y_test[i][lab] += ((log_sum_summary))
                
        
        for i in range(X_test.shape[0]):
            y_test[i] /= (np.sum(y_test[i]) - y_test[i])
            y_test[i] = -y_test[i]
            
        ret_val = np.zeros(len(y_test))
        
        cnt = 0
        for i in range(X_test.shape[0]):
            mx = 0
            mxindex = -1
            for j in range(5):
                if y_test[i][j] != 0:
                    if mxindex == -1 or y_test[i][j] > mx:
                        mxindex = j
                        mx = y_test[i][j]
            if mxindex == -1:
                cnt += 1
                mxindex = cnt % 5
            ret_val[i] = mxindex
        ret_val =  [int(round(x)) for x in ret_val] 
        print('Number of random classifications: ',cnt)
        return list(ret_val)

In [44]:

# create training and testing vars
y = df.loc[:,'label'] - 1
X = df.loc[:, 'summary':'component']

In [45]:
naiveBayesClassifier = NaiveBayesClassifer()

In [46]:
f1_scores = np.zeros((10, 5))
f1_scores_list = []
for i in range(1,11):
    split_point = (len(X)/11) * i
    naiveBayesClassifier.train(X_train = X.loc[0:split_point,], y_train = y.loc[0:split_point, ])
    y_test = naiveBayesClassifier.predict(X_test = X.loc[split_point:, ])
    
    
    y_label = list(y.loc[split_point:, ])
    
    #print(y_test, y_label)
    #print((y_test == y_label).count(True) / len(y_test))
    
    
    true_positive = np.zeros(5)
    
    false_positive = np.zeros(5)
    false_negative = np.zeros(5)
    
    for j in range(len(y_test)):
        if (y_test[j] == y_label[j]):
            true_positive[y_test[j]] += 1
        else:
            false_positive[y_test[j]] += 1
            false_negative[y_label[j]] += 1
    
    precision = np.zeros(5)
    recall = np.zeros(5)
    for j in range(5):
        precision[j] = true_positive[j] / (true_positive[j] + false_positive[j])
        recall[j] = true_positive[j] / (true_positive[j] + false_negative[j])
    
    for j in range(5):
        f1_scores[i-1][j] = (2 * (precision[j] * recall[j])) / (precision[j] + recall[j])
        if np.isnan(f1_scores[i-1][j]):
            f1_scores[i-1][j] = 0
    print('Count: Train vs Test')
    print(y_test.count(0), y_label.count(0))
    print(y_test.count(1), y_label.count(1))
    print(y_test.count(2), y_label.count(2))
    print(y_test.count(3), y_label.count(3))
    print(y_test.count(4), y_label.count(4))
    
    print('F-measure after ', i, 'th iteration: ', f1_scores[i-1])
    f1_scores_list.append(f1_scores)
    pass
    

Number of unique words: 2373
Total number of features: 2373
Number of selected features: 669
Number of random classifications:  0
Count: Train vs Test
200 278
358 459
797 643
669 612
371 403
F-measure after  1 th iteration:  [0.08368201 0.4871481  0.33333333 0.28259173 0.21447028]
Number of unique words: 3528
Total number of features: 3528
Number of selected features: 1264
Number of random classifications:  0
Count: Train vs Test
138 271
560 428
697 567
484 544
276 345
F-measure after  2 th iteration:  [0.10757946 0.44534413 0.34018987 0.38521401 0.22866345]
Number of unique words: 4802
Total number of features: 4802
Number of selected features: 1822
Number of random classifications:  0
Count: Train vs Test
144 264
324 403
704 518
543 456
201 275
F-measure after  3 th iteration:  [0.08823529 0.51169188 0.37479542 0.42042042 0.23529412]
Number of unique words: 5619
Total number of features: 5619
Number of selected features: 2182
Number of random classifications:  0
Count: Train vs Test


In [48]:
np.mean(np.array(f1_scores_list[0]), axis = 0)

array([0.25771319, 0.51251246, 0.40364033, 0.39472628, 0.19821954])

In [9]:
# Eclipse: 
# round robin classification:           [0.16300248, 0.17159368, 0.26298052, 0.1744121 , 0.12709789]
# without prior without smoothing:      [0.21465332, 0.10047268, 0.05816234, 0.08378671, 0.09948181]
# with prior without smoothing:         [0.17769152, 0.12833085, 0.14291655, 0.10463765, 0.0911579 ]
# without prior with smoothing:         [0.21558835, 0.23368606, 0.42980058, 0.31752347, 0.2647105 ]
# with prior with smoothing:            [0.0466586 , 0.21333919, 0.50649528, 0.30572038, 0.20783305]
# with likelihood ratio with smoothing: [0.21558835, 0.23368606, 0.42980058, 0.31752347, 0.2647105 ]
# p-value > .9 (1062):                  [0.21582993, 0.23066638, 0.42998575, 0.30339254, 0.24177725]

# Mozilla:
# critval - description:                [0.25771319, 0.51251246, 0.40364033, 0.39472628, 0.19821954]


In [ ]:
[0.07925408 0.00615385 0.01660735 0.25118036 0.17888563]